In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
sp = pd.read_csv('../match/cp.csv')

In [3]:
def get_concentracion_from_desc_comer(row):
    desc_comer = row['descripcion_comercial']
    #------------------------------4------------------------------#
    #afinitor 10 mg  30 comprimidos (everolimus)
    expresion_1 = '\d+\.?\d*\s*mg\s*\d{1,3}\s*(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral)'
    #giotrif 30mg (afatinib) 30 tabletas (medicamento de uso humano)
    expresion_1_1 = '\d+\.?\d*\s*mg.{0,40}\d{1,4}\s*(tabletas|capsulas|comprimido|solucion inyectable|solucion oral)' #menos exacta que la expresion_1
    #prycel (dasatinib) frasco con  60 tabletas de 70 mg (medicamento de uso humano)
    expresion_2 = '\d{1,3}\s*(tablet|capsulas|comprimidos|solucion inyectable|viales|solucion oral).{0,35}\d+\.?\d*\s*(mg|gramos)'#se modificó para que detecte gramos y viales
    #tabletas.{0,5}\d+\.?\d*\smg.{0,5}\d+
    expresion_2_2 = 'tabletas.{0,5}\d+\.?\d*\smg.{0,5}\d+'#Nueva
    #------------------------------3------------------------------#
    #avastin  100 mg/4 ml solucion inyectable (bevacizumab)
    expresion_soluciones = '\d+\.?\d*\s*mg.{0,30}\s*(solucion inyectable|solucion oral)'
    #mvasi (bevacizumab) ampula con 100 mg/ 4.0 ml (medicamento de uso humano)
    expresion_4 = '(frasco ampula|vial|ampula|jeringa)\s*.{0,15}\d+\.?\d*\s*mg'
    #ninlaro (ixazomib) caja colectiva con 3 cajas con una capsula de 2.3 mg (medicamento de uso humano)
    expresion_4_2 = '(1|un|una)*\s*(capsu|capsula|tabletas).{0,5}\d+\.?\d*\s*mg'
    #kadcyla 100mg frasco ampula (trastuzumab emtansina)
    expresion_6 = '\d+\.?\d*\s*mg.{0,18}(frasco ampula|vial|ampula|jeringa)'
    #medicamento de uso humano vitrakvi (larotrectinib 100mg)capsulas
    expresion_7 = '\d+\.?\d*\s*mg\)*capsulas'
    #tarceva 100 mg comprimidos  (erlotinib)
    expresion_3 = '\d+\.?\d*\s*mg\s*(tabletas|tablet|capsul|comprimidos|solucion inyectable|solucion oral)'
    #------------------------------2------------------------------#
    #kyprolis 60mg (carfilzomib) (medicamento de uso humano)
    expresion_5 = '\d+\.?\d*\s*mg'
    #vial solucion estabilizadora iv para blinatumomab caja con 12 viales
    expresion_5_2 = '\d+\s*(tabletas|tab|capsulas|capsula|comprimidos|solucion inyectable|solucion oral|vial)'
    #brigatinib (alunbring)(taqbletas)
    expresion_s_ff = '(tabletas|tab|capsulas|capsula|comprimidos|solucion inyectable|solucion oral|vial)'

    if not isinstance(desc_comer, str):
        return [np.nan, np.nan,np.nan, np.nan,0]
     #----------------------------------------4--------------------------------------#
    match = re.search(expresion_1, desc_comer)#'\d+\.?\d*\s*mg\s*\d{1,3}\s*(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral)'
    if match:#10 mg  30 comprimidos
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = "mg"
        p_ff = re.search("\d{1,3}\s*(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral)", info).group()
        presentacion =  re.search("\d{1,3}", p_ff).group()
        forma_farma = re.search("(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral)", p_ff).group()
        forma_farma = 'tabletas' if forma_farma == 'tab' else forma_farma
        return [concentracion, unidad_concentracion, presentacion, forma_farma,4]

    match = re.search(expresion_1_1, desc_comer)#'\d+\.?\d*\s*mg.{0,40}\d{1,4}\s*(tabletas|capsulas|comprimidos|solucion inyectable|solucion oral)'
    if match:#100 mg/4 ml solucion inyectable
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = "mg"
        p_ff = re.search("\d{1,4}\s*(tabletas|capsulas|comprimido|solucion inyectable|solucion oral)", info).group()
        presentacion =  re.search("\d{1,4}", p_ff).group()
        forma_farma = re.search("(tabletas|capsulas|comprimido|solucion inyectable|solucion oral)", p_ff).group()
        forma_farma = 'comprimidos' if  'comrpimido' in forma_farma else forma_farma
        return [concentracion, unidad_concentracion, presentacion, forma_farma,4]

    match = re.search(expresion_2, desc_comer)#'\d{1,3}\s*(tablet|capsulas|comprimidos|solucion inyectable|viales|solucion oral).{0,35}\d+\.?\d*\s*(mg|gramos)'
    if match:#60 capsulas de 250mg
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*(mg|gramos)", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = re.search("(mg|gramos)", c_uc).group()
        p_ff = re.search("\d{1,3}\s*(tablet|capsulas|comprimidos|solucion inyectable|viales|solucion oral)", info).group()
        presentacion = re.search("\d{1,3}", p_ff).group()
        forma_farma = re.search("(tablet|capsulas|comprimidos|solucion inyectable|viales|solucion oral)", p_ff).group()
        forma_farma = 'tabletas' if 'tablet' in forma_farma else forma_farma
        return [concentracion, unidad_concentracion, presentacion, forma_farma,4]

    match = re.search(expresion_2_2, desc_comer) #'tabletas.{0,5}\d+\.?\d*\smg.{0,5}\d+'
    if match:#tabletas de 250 mg con 30 
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = 'mg'
        p_ff = re.search("con\s*\d{1,3}", info).group()
        presentacion = re.search("\d{1,3}", p_ff).group()
        forma_farma = 'tabletas'
        return [concentracion, unidad_concentracion, presentacion, forma_farma,4]
    
    #----------------------------------------3--------------------------------------#
    match = re.search(expresion_3, desc_comer)#'\d+\.?\d*\s*mg\s*(tabletas|tablet|capsul|comprimidos|solucion inyectable|solucion oral)'
    if match:#250 mg tabletas
        info = match.group()
        numeros =  re.findall("\d+\.?\d*", info)
        concentracion = numeros[0]
        unidad_concentracion = 'mg'
        presentacion = 'sp'
        forma_farma = re.search("(tabletas|tablet|capsul|comprimidos|solucion inyectable|solucion oral)", info).group()
        forma_farma = 'tabletas' if forma_farma == 'tablet' else forma_farma
        forma_farma = 'capsulas' if 'capsul' in forma_farma else forma_farma
        return [concentracion, unidad_concentracion, presentacion, forma_farma,3]

    match = re.search(expresion_soluciones, desc_comer)#'\d+\.?\d*\s*mg.{0,30}\s*(solucion inyectable|solucion oral)'
    if match:#100 mg/4 ml solucion inyectable
        info = match.group()
        concentracion = re.search("\d+\.?\d*", info).group()
        unidad_concentracion = 'mg'
        presentacion = '1'
        forma_farma = re.search("(tabletas|tab|capsulas|comprimidos|solucion inyectable|solucion oral)", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma,4]
    
    match = re.search(expresion_4, desc_comer)#'(frasco ampula|vial|ampula|jeringa)\s*.{0,15}\d+\.?\d*\s*mg'
    if match:#ampula con 100 mg  
        info = match.group()
        c_uc = re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = 'mg'
        presentacion = '1'
        forma_farma = re.search("(frasco ampula|vial|ampula|jeringa)", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma,4]

    match = re.search(expresion_4_2, desc_comer)#'(1|un|una)*\s*(capsu|capsula|tabletas).{0,5}\d+\.?\d*\s*mg'
    if match:#una capsula de 2.3 mg
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = "mg"
        p_ff = re.search("(1|un|una)*\s*(capsu|capsula|tabletas)", info).group()
        presentacion =  re.search("(1|un|una)", p_ff)
        forma_farma = re.search("(capsu|capsula|tabletas)", p_ff).group()
        forma_farma = 'capsulas' if (forma_farma == 'capsu' or forma_farma == 'capsula') else forma_farma
        presentacion = '1' if presentacion else 'sp'
        caracteristicas = 4 if presentacion == '1' else 3
        return [concentracion, unidad_concentracion, presentacion, forma_farma,caracteristicas]

    match = re.search(expresion_6, desc_comer)#\d+\.?\d*\s*mg.{0,10}(frasco ampula|vial|ampula|jeringa)
    if match:#100mg frasco ampula #120 mg/ 2.4 ml frasco ampula
        info = match.group()
        c_uc =  re.search("\d+\.?\d*\s*mg", info).group()
        concentracion = re.search("\d+\.?\d*", c_uc).group()
        unidad_concentracion = "mg"
        presentacion = '1'
        forma_farma = re.search("(frasco ampula|vial|ampula|jeringa)", info).group()
        return [concentracion, unidad_concentracion, presentacion, forma_farma,4]
    
    match = re.search(expresion_7, desc_comer)#\d+\.?\d*\s*mg\)*capsulas
    if match: #100mg)capsulas
        info = match.group()
        concentracion = re.search("\d+\.?\d*", info).group()
        unidad_concentracion = 'mg'
        presentacion = 'sp'
        forma_farma = 'capsulas'
        return [concentracion, unidad_concentracion, presentacion, forma_farma,3]
   
    #----------------------------------------2--------------------------------------#
    match = re.search(expresion_5, desc_comer)#\d+\.?\d*\s*mg
    if match:#250 mg
        info = match.group()
        concentracion =  re.search("\d+\.?\d*", info).group()
        unidad_concentracion = 'mg'
        presentacion = 'sp'
        forma_farma = 'sff'
        return [concentracion, unidad_concentracion, presentacion, forma_farma,2]
    
    match = re.search(expresion_5_2, desc_comer)#\d+\s*(tabletas|tab|capsulas|capsula|comprimidos|solucion inyectable|solucion oral|vial)
    if match:#12 viales
        info = match.group()
        concentracion = 'sc'
        unidad_concentracion = 'suc'
        presentacion = re.search("\d+", info).group()
        forma_farma = re.search("(tabletas|tab|capsulas|capsula|comprimidos|solucion inyectable|solucion oral|vial)", info).group()
        forma_farma = 'tabletas' if forma_farma == 'tab' else forma_farma
        forma_farma = 'capsulas' if 'capsul' in forma_farma else forma_farma
        return [concentracion, unidad_concentracion, presentacion, forma_farma,2]
    
    #----------------------------------------1--------------------------------------#
    match = re.search(expresion_s_ff, desc_comer)#(tabletas|taqbletas|tab|capsulas|capsula|comprimidos|solucion inyectable|solucion oral|vial)
    if match:#taqbletas
        info = match.group()
        concentracion = 'sc'
        unidad_concentracion = 'suc'
        presentacion = 'sp'
        forma_farma = match.group()
        forma_farma = 'tabletas' if ('tab' in forma_farma or forma_farma == 'taqbletas') else forma_farma
        forma_farma = 'capsulas' if 'capsul' in forma_farma else forma_farma
        return [concentracion, unidad_concentracion, presentacion, forma_farma,1]

    return [np.nan, np.nan, np.nan, np.nan, 0]

In [4]:
sp[['concentracion','unidad_concentracion','presentacion','forma_farma','caracteristicas']] = sp.apply(get_concentracion_from_desc_comer,axis=1,result_type='expand')